In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import matplotlib.pyplot as plt 

import os

import numpy as np

from PIL import Image
from PIL import ImageOps

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as TRS

%matplotlib inline


import time

In [ ]:
!pip install mtgsdk

In [ ]:
# Fonctions pour l'API MTG

from mtgsdk import Card
from mtgsdk import Set
from mtgsdk import Type
from mtgsdk import Supertype
from mtgsdk import Subtype
from mtgsdk import Changelog

from random import shuffle


import urllib.request

In [ ]:
!pip install praw
!pip install wget

In [ ]:
# Fonctions pour l'API Reddit
import re
import praw
import wget
import shutil

La partie qui suit est consacré au téléchargement et au traitement des cartes. C'est vraiment pas une manière de procéder, mais c'est le code que j'avais utilisé à l'époque, et en toute honnêteté, j'ai pas envie de me consacrer de trop à la partie secondaire. 
Dans les trucs à amméliorer pour le dataset, il y a se démerder pour que les images soient toujours différentes (ce qui nécessiterait aussi de faire un croismenet avec les images du INE), et il faudrait pouvoir croper un peu mieux les images qui sont pénibles (PW, Schemes...)

In [ ]:
BOX = (28, 38, 196, 166)
SIZE = (168,128)
FACTOR = 0.3

# Idéalement, faudrait aller chercher la liste sur le site
INE_SUBS = [#"Angels",
            #"Immortals",
            #"Battlefields",
            #"Cityscapes",
            #"Mindscapes",
            #"Pathways",
            #"Seascapes",
            #"Skyscapes",
            #"Starscapes",
            #"Wastelands",
            #"Weather",
            #"Wildlands",
            #"Worlds",
            #"Architecture",
            #"Castles",
            #"Dwellings",
            #"Interiors",
            #"Libraries",
            #"Beasts",
            #"Behemoths",
            #"Carnage",
            #"Dragons",
            #"Demons",
            #"Elementals",
            #"Horrors",
            #"Hybrids",
            #"Leviathans",
            #"Undead",
            #"WorldEaters",
            #"FutureWar",
            #"Futurism",
            #"Portals",
            #"Starships",
            #"Steampunk",
            #"Gatherings",
            #"Autumnscapes",
            #"Bodyscapes",
            #"Fogscapes",
            #"Nightscapes",
            #"Springscapes",
            #"Summerscapes",
            #"Waterscapes",
            #"Winterscapes",
            #"Canyons",
            #"Caves",
            #"Deserts",
            #"Forests",
            #"Glaciers",
            #"Islands",
            #"Lakes",
            #"Mountains",
            #"Rivers",
            #"Swamps",
            #"Trees",
            #"Volcanoes",
            #"Waterfalls",
            #"Asylums",
            #"Factories",
            #"Monuments",
            #"Prisons",
            #"Ruins",
            #"Statues",
            #"Taverns",
            #"Temples",
            #"Towers",
            #"Villages",
            #"Walls",
            #"Crawlers",
            "Dinosaurs",
            "Spirits",
            "Unicorns",
            "Vampires",
            "Werewolves",
            ]
OTHER_SUBS = ["CelestialBodies",
              ]
SUBS = ["Imaginary{}".format(ine) for ine in INE_SUBS] +  OTHER_SUBS

main_path = os.path.join(os.getcwd(),"drive","My Drive", "CARDS")



In [ ]:

def dlsets(sets = None, start_set = None, stop_set = None, path = os.path.join(main_path,"Cards")):
    '''
    Cette fonction télécharge dans le dossier Cards toutes les cartes de tous les sets dans sets à partir du set start_set et jusqu'au set stop_set inclus.
    '''
    if sets == None:
        sets = Set.all()
    if start_set == None:
        dl = True
    else:
        dl = False
    try:
        os.mkdir(os.path.join(main_path,"Cards"))
    except:
        pass
 
    for set in sets:
        if set.name == start_set:
            dl = True
        if dl:
            dlset(set, path)
        else:
            print("Skipping {}".format(set.name))
        if set.name == stop_set:
            dl = False

def dlone(set):
    dlsets(start_set = set, stop_set = set)

def dlset(set, path):
    print("Téléchargement du set : " + set.name)
    cards = Card.where(set=set.code).all()
    for card in cards:
        if card.image_url != None:
            dlcard(card, path)

def dlcard(card, path):
    print("Téléchargement de la carte : " + card.name + " (" + str(card.multiverse_id) + ")")
    if card.image_url == None:
        print("Erreur dans le téléchargement, la carte n'a pas d'illustration.")
        return
    try:
        urllib.request.urlretrieve(card.image_url, os.path.join(path, str(card.multiverse_id)))
    except:
        print("Attention : une carte n'a pas été téléchargée avec l'ID {}".format(id))

def crop_cards(selected_cards = None, start_card = None):
    try:
        os.makedirs(os.path.join(main_path,"Cards_croped"))
    except:
        print("Failed to create file for croped cards, maybe there is already one")
        pass
    cards = os.listdir(os.path.join(main_path,"Cards"))
    if start_card != None:
        cards = cards[cards.index(start_card)+1:]
    for card in cards:
        if selected_cards == None or card in selected_cards:
            crop_one(card)

def resize_cards(selected_cards = None, start_card = None):
    try:
        os.mkdir(os.path.join(main_path,"Cards_resized"))
    except:
        pass
    cards = os.listdir(os.path.join(main_path,"Cards_croped"))
    if start_card != None:
        cards = cards[cards.index(start_card)+1:]
    for card in cards:
        if selected_cards == None or card in selected_cards:
            resize_one(card)

def resize_one(card,path_to_crops = os.path.join(main_path,"Cards_croped"), path_to_resize = os.path.join(main_path,"Cards_resized")):
    print("Resize de la carte :" + str(card))
    try:
        picture = Image.open(os.path.join(path_to_crops, card))
        resize = picture.resize(SIZE)
    except:
        pass
    resize.save(os.path.join(path_to_resize,card), format = picture.format)

def crop_one(card, path_to_crops = os.path.join(main_path,"Cards_croped"), path_to_cards = os.path.join(main_path,"Cards")):
    print("Crop de la carte : " + str(card))
    try:
        picture = Image.open(os.path.join(path_to_cards, card))
        crop = picture.crop(BOX)
    except OSError:
        print("OSError, peut-être la carte suivante est absente : " + str(card))
        card2 = Card.find(card)
        dlcard(card2, path_to_cards)
        picture = Image.open(os.path.join(path_to_cards, card))
        crop = picture.crop(BOX)
    crop.save(os.path.join(path_to_crops, card), format = picture.format)

def label_cards(path_to_cards = os.path.join(main_path,"Cards_resized"), path_to_labeled = os.path.join(main_path,"Cards_labeled")):
    try:
        os.makedirs(path_to_labeled)
    except:
        pass
    cards = os.listdir(path_to_cards)
    shuffle(cards)
    for card in cards:
        label_one(card, path_to_cards = path_to_cards, path_to_labeled = path_to_labeled)

def label_one(card, path_to_cards, path_to_labeled):
    print("Labeling : ", card)
    try:
        id = int(card)
    except:
        print("Id incompatible :", card)
        return
    #picture = Image.open(os.path.join(path_to_cards, card))
    labeled = os.listdir(path_to_labeled)
    for label in labeled:
        if label[:len(card)] == card and label[len(card)] == "C":
            color = label[label.find("C"):]
            print("Carte déjà présente avec l'identité couleur : ", color)
            #picture.save(os.path.join(path_to_labeled, card + color), format = picture.format)
            return
    j = 0
    card_gatherer = Card.find(id)
    while 1:
        try:
            card_gatherer = Card.find(id)
        except:
            j += 1
            if j==1:
                print("Échec pour étiqueter la carte, elle ne sera pas ajouté au set de données.")
                return
    color_identity = card_gatherer.color_identity
    if color_identity == None:
        color = "C"
    else:
        color = "C"+"".join(color_identity)
    print("Color identityt is {}".format(color_identity))
    open(os.path.join(path_to_labeled,card + color), 'a').close()


def fetch_repartition(path_to_cards = os.path.join(main_path,"Cards_labeled")):
    cards = os.listdir(path_to_cards)
    result = dict()
    for card in cards:
        color = card[card.find("C"):]
        color_index = 1
        colors = ["W","U","B","R","G"]
        prime = [2,3,5,7,11]
        for i in range(5):
            if colors[i] in color:
                color_index *=prime[i]
        color = ""

        for i in range(5):
            if color_index%prime[i] == 0:
                color += colors[i]
        try:
            result[color]+=1
        except:
            result[color] =1
    return result


In [ ]:
# Cette fonction n'a pas de manière de faire de récupération de ce qui a déjà 
# été téléchargé, mais elle peut reprendre un téléchargement en cours en 
# spécifiant le nom du set de départ

dlsets()

crop_cards()

resize_cards()


# La fonction suivante gère assez mal les images qui ne peuvent pas être 
# étiquettée : elle essaye une ou deux fois et après dit que l'image ne devait 
# pas correspondre à une carte. L'exécution prend du temps même si tout a déjà 
# été fait, et ne fournit même pas l'assurance d'étiqueter tout ce qui est 
# possible.

label_cards()

Cette partie est consacrée à télécharger les fichiers du INE et de les transformer et les découper en conséquences.

In [ ]:
checkWords = ['i.imgur.com',  'jpg', 'png', 'gif', 'gfycat.com', 'webm',]

In [ ]:
def download_INE(subs = SUBS, path_to_rip = "rips"):
    try:
        os.makedirs(path_to_rip)
    except:
        pass
    with open(os.path.join(main_path, "reddit.id"), "r") as file_info_reddit:
        info_reddit = file_info_reddit.read().split("\n")
        r = praw.Reddit(client_id = info_reddit[0], client_secret= info_reddit[1], user_agent = "INE dl")
    for sub in subs:
        print("Téléchargement des fichiers du Sub {}".format(sub))
        download_sub(r, sub, path_to_rip = path_to_rip)
        convert_and_resize(path_to_rip = path_to_rip)
        shutil.rmtree(os.path.join(path_to_rip, sub))


def download_sub(r, sub, path_to_rip =  "rips"):
    path_to_sub_file = os.path.join(path_to_rip, sub)
    try:
        os.makedirs(path_to_sub_file)
    except:
        pass
    subreddit = r.subreddit(sub)
    for submission in subreddit.top(limit = 1000):
        url_text = submission.url
        has_domain = any(string in url_text for string in checkWords)
        print('[LOG] Getting url:  ' + url_text)
        if has_domain and submission.score >0 and submission.upvote_ratio > 0.5:
            try:
                wget.download(url_text, os.path.join(path_to_rip,sub ,sub + str(time.time())[-8:-3] + url_text[-4:]))
                #print('[LOG] Done Getting ' + url_text)
            except:
                #print("Can't download from location : {}".format(url_text))
                try:
                    wget.download(submission.preview["images"][0]["source"]["url"],os.path.join(path_to_rip,sub ,sub + str(time.time())[-8:-3] + url_text[-4:]))
                    #print('[LOG] Done Getting from preview :' + submission.preview["images"][0]["source"]["url"])
                except:
                    print("Can't download from reddit preview or from direct source: {}".format(str(url_text)))

def convert_and_resize(path_to_rip = "rips", path_to_resize = os.path.join(main_path, "rips_resized")):
    try:
        os.makedirs(path_to_resize)
    except:
        pass
    files = os.listdir(path_to_rip)
    for file in files:
        print("Processing : ", file)
        try:
            pictures = os.listdir(os.path.join(path_to_rip,file))
        except os.NotADirectory:
            print("{} is not a valid directory.".format(file))
        for picture in pictures:
            if picture[-4:] in [".png",".jpg"]:
                convert_and_resize_one(os.path.join(path_to_rip,file, picture), path_to_resize = path_to_resize, name = picture)

def convert_and_resize_one(path_to_picture, path_to_resize = os.path.join(main_path, "rips_resized"), name = "test.png"):
    print("Converting :", path_to_picture)
    try:
        picture = Image.open(path_to_picture)
    except:
        print("Error : cant open file :", path_to_picture)
        return
    already_resized = os.listdir(path_to_resize)
    if picture.size[0]/picture.size[1] > SIZE[0]/SIZE[1]:
        factor = SIZE[1]/picture.size[1]
        picture = picture.resize((int(picture.size[0] * factor), SIZE[1]))
        picture = picture.crop((picture.size[0]//2 - SIZE[0]//2,0, picture.size[0]//2 + SIZE[0]//2 + int(SIZE[0]%2 ==1),SIZE[1]))
    else:
        factor = SIZE[0]/picture.size[0]
        picture = picture.resize((SIZE[0], int(picture.size[1] * factor)))
        if picture.size[1] * FACTOR <  SIZE[1]//2:
            picture = picture.crop((0,0,SIZE[0],SIZE[1]))
        else:
            picture = picture.crop((0,int(picture.size[1] * FACTOR) - SIZE[1]//2,SIZE[0],int(picture.size[1]* FACTOR) + SIZE[1]//2 + int(SIZE[1]%2 ==1)))
    assert picture.size == SIZE
    try:
        picture.save(os.path.normpath(os.path.join(path_to_resize,name)), format = picture.format)
    except:
        print("Error : cant save file:", path_to_picture)
        return

In [ ]:
# Cette fonction est principalement récupérée depuis :
# https://github.com/IeuanG/mxtm/blob/master/main.py
download_INE(subs = SUBS)


#
#convert_and_resize()

In [ ]:
len(os.listdir(os.path.join(main_path, "rips_resized")))

In [ ]:
len(os.listdir(os.path.join(main_path, "Cards_resized")))

In [ ]:
batch_size = 10

Un petit paquet de foncitons utilitaires:

In [ ]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    print("GPU utilisable")
    
def gpu(tensor, gpu=use_gpu):
    if gpu:
        return tensor.cuda()
    else:
        return tensor
     
def show(img):
    npimg = img.numpy()
    plt.figure(figsize = (20,20))
    plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')
    
def sigma(x):
    return torch.clamp(x,0,1)

def onehot_from_label(x, label_dim = 10):
    label_onehot = gpu(torch.FloatTensor(x.size()[0], label_dim))
    label_onehot.zero_()
    label_onehot.scatter_(1,x.long().view(-1,1), 1)
    return label_onehot

def random_label(batch_size = batch_size, label_dim = 5, multi_label = False):
    if multi_label:
        pass
    else:
        return onehot_from_label(gpu(torch.randint(high = label_dim, size = (batch_size,))))

def test_label(batch_size = batch_size, label_dim = 5):
    return onehot_from_label(gpu(torch.Tensor([i%label_dim for i in range(batch_size)])))

def index_to_colors(l):
    result = ""
    for i in range(5):
        if l[i]>0.5:
            result += COLORS[i]
        elif l[i]>0.1:
            result += COLORS[i].lower()
    return result



def show_loss(loss_G_list, loss_D_list, loss_C_list = None):
    plt.figure()
    plt.plot(list(range(len(loss_G_list))),loss_G_list, label = "generator")
    plt.show()
    plt.figure()
    plt.plot(list(range(len(loss_D_list))),loss_D_list, label = "discriminator")
    plt.show()
    if loss_C_list != None:
        plt.figure()
        plt.plot(list(range(len(loss_C_list))),loss_C_list, label = "classifier")
        plt.show()
        
def test_D(net_C, batch,n = 5):
    out = batch.view(-1, 3, 128, 168)[:n]
    out_grid = torchvision.utils.make_grid(out, nrow = 5).cpu()
    result = net_C(batch)
    for i in range(min(n,len(result))):
        print(i, index_to_colors(result[i]))
    show(sigma(out_grid))
    
    
def show_exemple(net_G, size1 = 28, size2 = 28, n_label = 5, n_row = 5, channel = 1, max_n = 20, test_mode = False):
    if test_mode:
        n_row = n_row//(T+1)
    if use_C:
        label = test_label(label_dim = n_label)
        with torch.no_grad():
            out = net_G(label = label, test_mode = test_mode).view(-1, channel, size1, size2*(T+1))[:max_n]
        out_grid = torchvision.utils.make_grid(out, nrow = n_row).cpu()
    else:
        with torch.no_grad():
            out = net_G(test_mode = test_mode).view(-1, channel, size1 , size2 * (T+1))[:max_n]
        out_grid = torchvision.utils.make_grid(out, nrow = n_row).cpu()

    show(sigma(out_grid))

Maintenant, il s'agit de faire les datasets.

In [ ]:
class UnlabeledData(torch.utils.data.Dataset):
    def __init__(self, path_to_pictures):
        self.cards = os.listdir(path_to_pictures)
        self.path_to_pictures = path_to_pictures

    def __getitem__(self,index):
        if index > len(self.cards):
            mirror = True
        else:
            mirror = False
        img = Image.open(os.path.join(self.path_to_pictures,self.cards[index - int(mirror)*len(self.cards)]))
        if mirror:
            img2 = ImageOps.mirror(img)
        else:
            img2 = img
        img2 = img2.convert("RGB")
        img.close()
        img = np.asarray(img2, dtype = "float").transpose(-1, 0, 1)
        img = torch.from_numpy(np.asarray(img, dtype = "float")).float()
        return img/255

    def __len__(self):
        return len(self.cards)

class LabeledData(torch.utils.data.Dataset):
    def __init__(self, path_to_pictures):
        self.cards = os.listdir(path_to_pictures)
        self.path_to_pictures = path_to_pictures

    def __getitem__(self,index):
        if index > len(self.cards):
            mirror = True
        else:
            mirror = False
        card = self.cards[index- int(mirror)*len(self.cards)]
        img = Image.open(os.path.join(self.path_to_pictures,card))
        if mirror:
            img2 = ImageOps.mirror(img)
        else:
            img2 = img
        img2 = img2.convert("RGB")
        img.close()
        img = np.asarray(img2,dtype = "float").transpose(-1, 0, 1)
        img = torch.from_numpy(np.asarray(img)).float()
        label = torch.zeros(5)
        color = ["W","U","B","R","G"]
        for i in range(len(color)):
            if color[i] in card:
                label[i] = 1.
        return img/255,label

    def __len__(self):
        return len(self.cards)

In [ ]:
unlabeled_set = UnlabeledData(os.path.join(main_path, "rips_resized"))
unlabeled_loader = torch.utils.data.DataLoader(unlabeled_set, batch_size = batch_size, shuffle = True, drop_last = True)

On passe maintenant au duo generateur / discrimateur.
On veut utiliser le papier suivant : https://arxiv.org/pdf/1912.04958.pdf.